# ECE 276 Project 2: LiDAR based SLAM

1. load data

In [86]:
!pip install open3d matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt; plt.ion()
import time
import os
import scipy.io as sio
import open3d as o3d
import gtsam

from icp_warm_up.utils import read_canonical_model, load_pc, visualize_icp_result
import pr2_utils as U
from load_data import load_dataset


print("done")


done


In [ ]:
def encoder_IMU_odometry(dataset = 20, d_tick = 0.0022):
    encoder_data, lidar_data, imu_data, kinect_data = load_dataset(dataset)
    encoder_counts = encoder_data["encoder_counts"]
    encoder_stamps = encoder_data["encoder_stamps"] # 40 Hz 
    
    imu_angular_velocity = imu_data["imu_angular_velocity"] 
    imu_stamps = imu_data["imu_stamps"] # 100 Hz 
    print("encoder stamps size", len(encoder_stamps)) 
    print("imu stamps size", len(imu_stamps)) 
    
    # only use time range when both sensors are on 
    t_start = max(encoder_stamps[0], imu_stamps[0]) 
    t_end = min(encoder_stamps[-1], imu_stamps[-1]) 
    print("\n\ntime range", t_start, "to ", t_end) 


    imu_range = (imu_stamps >= t_start) & (imu_stamps <= t_end)
    
    imu_angular_velocity_yaw = imu_angular_velocity[2, imu_range]               # imu yaw only synced max start, min end
    imu_stamps_yaw = imu_stamps[imu_range]                                      # sync up imu with average imu over longer encoder time steps since encoder is slower/less samples 

    encoder_range = (encoder_stamps >= t_start) & (encoder_stamps <= t_end)     # encoder to max start, min end 
    encoder_stamps = encoder_stamps[encoder_range]
    encoder_counts = encoder_counts[:, encoder_range]

    encoder_dt = np.diff(encoder_stamps) 
    
    # encoder displacements 
    d_R = (encoder_counts[0,:] + encoder_counts[2,:]) * d_tick/2                # (FR+RR)/2 * 0.0022m 
    d_L = (encoder_counts[1,:]+encoder_counts[3,:]) * d_tick/2                  # (FL+RL)/2 * 0.0022m 
    d_center = (d_R + d_L) / 2                                                  # distance traveled by robot center 
    
    v = d_center[1:]/encoder_dt # velocity at robot center 
    
    imu_dt = np.diff(imu_stamps_yaw) 
    imu_theta_yaw = np.zeros(imu_angular_velocity_yaw.shape) 
    imu_theta_yaw[1:] = np.cumsum((imu_angular_velocity_yaw[0:-1]+imu_angular_velocity_yaw[1:])/2 * imu_dt) # interporlate imu data instead of finding nearest points to encoder timestamp 
    
    encoder_theta = np.interp(encoder_stamps, imu_stamps_yaw, imu_theta_yaw) # interpolate imu angular distance over encoder timestamp 
    d_theta = np.diff(encoder_theta) 
    # omega = d_theta / encoder_dt 
    
    N = len(encoder_dt) 
    x = np.zeros(N+1) 
    y = np.zeros(N+1) 
    # theta = np.zeros(N+1) # use abosulte, dead reckon the changes in orientation every step 
    
    # theta[1:] = np.cumsum(omega * encoder_dt) 
    # for i in range(N):
    #     x[i+1] = x[i] + d_center[i] * np.cos(encoder_theta[i]) 
    #     y[i+1] = y[i] + d_center[i] * np.sin(encoder_theta[i]) 
    
    d_x = d_center[1:] * np.cos(encoder_theta[:-1]) # d_center = v * dt 
    d_y = d_center[1:] * np.sin(encoder_theta[:-1]) 
    
    x[1:] = np.cumsum(d_x) 
    y[1:] = np.cumsum(d_y)

    plt.figure()
    plt.plot(x, y)
    plt.axis("equal")
    plt.title("Part 1 Odometry Trajectory (Encoder + IMU)")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.grid(True)
    plt.show()

    return 



In [88]:
encoder_IMU_odometry(20)
encoder_IMU_odometry(20)

encoder stamps size 4956
imu stamps size 12187


time range 1298445274.823653 to  1298445396.755451
encoder stamps size 4956
imu stamps size 12187


time range 1298445274.823653 to  1298445396.755451
